In [38]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import Model
from keras.optimizers.legacy import Adam
from keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, TensorBoard

import certifi
import urllib3

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
response = http.request('GET', 'https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5')


In [8]:
train_path = 'archive/train'
test_path = 'archive/test'
valid_path = 'archive/valid'

image_size = (224, 224)
batch_size = 128
seed = 123567
data_augmentation = True


In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.15,
    horizontal_flip=True,
)

valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [7]:
eff_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)


In [34]:
train_df = train_generator.flow_from_directory(
    train_path,
    target_size=image_size,
    seed=seed,
    class_mode='categorical',
    color_mode= 'rgb',
    batch_size=batch_size,
    shuffle = True
)

validation_df = valid_generator.flow_from_directory(
    valid_path,
    target_size=image_size,
    seed=seed,
    class_mode='categorical',
    batch_size=batch_size,
    color_mode='rgb',
)
test_df = test_generator.flow_from_directory(
    test_path,
    target_size=image_size,
    seed = seed,
    class_mode = "categorical",
    color_mode= 'rgb',
    batch_size = batch_size
)

len(train_df)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


662

In [23]:
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

In [44]:

model_path = 'archive/EfficientNetB0-525.h5'
# Define the custom metric function
model= tf.keras.models.load_model(model_path, custom_objects={'F1_score':'F1_score'})
model.trainable = False

In [43]:
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)



pretrained_model.trainable = False

Exception: URL fetch failure on https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)

In [45]:
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 5 epochs in a row, stop training

# Setup decaying Learning rate on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Log data for TensorBoard
log_dir = "logs"  # Specify your log directory
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [46]:
inputs = model.input
x = augment(inputs)

x = Dense(128, activation='relu')(model.output)
x = Dropout(0.25)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)


outputs = layers.Dense(524, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [47]:
history = model.fit(
    train_df,
    steps_per_epoch=len(train_df),
    validation_data=validation_df,
    validation_steps=len(validation_df),
    epochs=15,
    callbacks=[
        early_stopping,
        tensorboard_callback,
        reduce_lr
    ]
)

Epoch 1/15


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/novailable/Aves_SSR/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/novailable/Aves_SSR/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/novailable/Aves_SSR/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/novailable/Aves_SSR/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

